## Fine Tuning Using Gemma Models

https://wandb.ai/

In [1]:
# 1. Downgrade transformers to a version compatible with TRL 0.7.10
!pip install -q --upgrade transformers==4.38.2

# 2. Ensure TRL is pinned correctly
!pip install -q --upgrade trl==0.7.10

# 3. Re-run the other pins to ensure consistency
!pip install -q --upgrade diffusers==0.27.2
!pip install -q --upgrade accelerate==0.27.2
!pip install -q --upgrade bitsandbytes==0.42.0
!pip install -q --upgrade peft==0.8.2
!pip install -q --upgrade sentence-transformers==2.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 9.2 MB/s et

In [2]:
!pip install --upgrade trl



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.27.2
    Uninstalling accelerate-0.27.2:
      Successfully uninstalled accelerate-0.27.2
  Attempting uninstall: trl
    Found existing installation: trl 0.7.10
    Uni

In [3]:
!pip install bitsandbytes --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.42.0
    Uninstalling bitsandbytes-0.42.0:
      Successfully uninstalled bitsandbytes-0.42.0


In [4]:
import torch
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

print("Imports successful!")

Imports successful!


In [5]:
import os
import transformers
import torch
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer


In [6]:
from google.colab import userdata
os.environ["HF_TOKEN"]=userdata.get("HF_TOKEN")

## Prerequisites

In [7]:
model_id="google/gemma-2b"
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
import bitsandbytes as bnb
import torch

# This should print True
print(f"CUDA Available: {torch.cuda.is_available()}")

# This should print the correct library path without crashing
print(bnb.__version__)

CUDA Available: True
0.48.2


In [9]:
from transformers.utils import quantization_config
tokenizer=AutoTokenizer.from_pretrained(model_id,token=os.environ['HF_TOKEN'])


model=AutoModelForCausalLM.from_pretrained(model_id,
                                           quantization_config=bnb_config,
                                           device_map={"":0},
                                           token=os.environ['HF_TOKEN'])

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [10]:
text="Quote:Imagination is more ,"
device="cuda:0"

inputs=tokenizer(text,return_tensors="pt").to(device)


outputs=model.generate(**inputs,max_new_tokens=50)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))



Quote:Imagination is more , than knowledge.

I am a self-taught artist, born in 1985 in the city of São Paulo, Brazil. I have a degree in Fine Arts from the University of São Paulo.

I have been painting for more than


In [11]:
os.environ["WANDB_DISABLED"]="false"

In [12]:
lora_config=LoraConfig(
    r=8,
    target_modules=["q_proj","o_proj","k_proj","v_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM"
)

In [13]:
from datasets import load_dataset
data=load_dataset("Abirate/english_quotes")

#data=data.map(lambda samples:tokenizer(samples["quote"]),batched=True)

README.md: 0.00B [00:00, ?B/s]

quotes.jsonl:   0%|          | 0.00/647k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [14]:
data['train']['quote']

Column(['“Be yourself; everyone else is already taken.”', "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”", "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.”", '“So many books, so little time.”', '“A room without books is like a body without a soul.”'])

In [15]:
data['train']['author']

Column(['Oscar Wilde', 'Marilyn Monroe', 'Albert Einstein', 'Frank Zappa', 'Marcus Tullius Cicero'])

In [20]:
# def formatting_func(example):
#   text=f"Quote: {example['quote'][0]}\nAuther:  {example['author'][0]}"
#   return [text]

# 2. Define the formatting function
def formatting_prompts_func(example):
    output_texts = []
    # Iterate over the batch
    for quote, author in zip(example['quote'], example['author']):
        # Format: Quote + Author
        text = f"Quote: {quote}\nAuthor: {author}"
        output_texts.append(text)
    # Return a dictionary to create the new 'text' column
    return {"text": output_texts}

data = data.map(formatting_prompts_func, batched=True)

# Verify it looks correct (should print a single string, not a list)
print("Sample format:", data['train'][0]['text'])


Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Sample format: Quote: “Be yourself; everyone else is already taken.”
Author: Oscar Wilde


In [21]:
data['train']

Dataset({
    features: ['quote', 'author', 'tags', 'text'],
    num_rows: 2508
})

In [24]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],  # <--- Tell trainer to use our manually created column
    # formatting_func=formatting_func,  <--- REMOVE THIS LINE to avoid the bug
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
)

Adding EOS to train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [25]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mohammadalghani61 (mohammadalghani61-boj) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,2.656600
2,2.092800
3,2.584900
4,2.823400
5,2.376600
6,2.439100
7,2.787400
8,2.056600
9,2.881800
10,2.212600


TrainOutput(global_step=100, training_loss=1.8059492844343186, metrics={'train_runtime': 146.3194, 'train_samples_per_second': 2.734, 'train_steps_per_second': 0.683, 'total_flos': 235740440186880.0, 'train_loss': 1.8059492844343186, 'epoch': 0.1594896331738437})

In [31]:
text="Quote: You only live once,"
device="cuda:0"

inputs=tokenizer(text,return_tensors="pt").to(device)


outputs=model.generate(**inputs,max_new_tokens=20)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Quote: You only live once, but if you do it right, once is enough.
Author: Mae West
